# Main built-in LCEL functions for runnables

## Contents
* .bind()
* .assign()

## Setup

#### After you download the code from the github repository in your computer
In terminal:
* cd project_name
* pyenv local 3.11.4
* poetry install
* poetry shell

#### To open the notebook with Jupyter Notebooks
In terminal:
* jupyter lab

Go to the folder of notebooks and open the right notebook.

#### To see the code in Virtual Studio Code or your editor of choice.
* open Virtual Studio Code or your editor of choice.
* open the project-folder
* open the 006-builtin-functions-for-runnables.py file

## Create your .env file
* In the github repo we have included a file named .env.example
* Rename that file to .env file and here is where you will add your confidential api keys. Remember to include:
* OPENAI_API_KEY=your_openai_api_key
* LANGCHAIN_TRACING_V2=true
* LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
* LANGCHAIN_API_KEY=your_langchain_api_key
* LANGCHAIN_PROJECT=your_project_name

We will call our LangSmith project **006-builtin-functions-with-runnables**.

## Connect with the .env file located in the same directory of this notebook

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [ ]:
#!pip install python-dotenv

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

#### Install LangChain

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [3]:
#!pip install langchain

## Connect with an LLM

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [4]:
#!pip install langchain-openai

* NOTE: Since right now is the best LLM in the market, we will use OpenAI by default. You will see how to connect with other Open Source LLMs like Llama3 or Mistral in a next lesson.

In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

## LCEL Chain

In [6]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [7]:
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

* The "pipe" operator `|` is the main element of the LCEL chains.
* The order (left to right) of the elements in a LCEL chain matters.
* An LCEL Chain is a Sequence of Runnables.

In [8]:
chain = prompt | model | output_parser

chain.invoke({"soccer_player": "Ronaldo"})

'One curious fact about Cristiano Ronaldo is that he has his own museum dedicated to his career and achievements in his hometown of Funchal, Madeira, Portugal. The museum showcases memorabilia, trophies, and other items related to his successful career in football.'

* All the components of the chain are Runnables.
* When we write chain.invoke() we are using invoke with all the componentes of the chain in an orderly manner:
    * First, we apply .invoke() to the prompt.
    * Then, with the previous output, we apply .invoke() to the model.
    * And finally, with the previous output, we apply .invoke() to the output parser.

## Use of .bind() to add arguments to a Runnable in a LCEL Chain
* For example, we can add an argument to stop the model response when it reaches the word "Ronaldo":

In [9]:
chain = prompt | model.bind(stop=["Ronaldo"]) | output_parser

In [11]:
chain.invoke({"soccer_player": "Ronaldo"})

'One curious fact about '

## Use of .bind() to call an OpenAI Function in a LCEL Chain

In [12]:
functions = [
    {
      "name": "soccerfacts",
      "description": "Curious facts about a soccer player",
      "parameters": {
        "type": "object",
        "properties": {
          "question": {
            "type": "string",
            "description": "The question for the curious facts about a soccer player"
          },
          "answer": {
            "type": "string",
            "description": "The answer to the question"
          }
        },
        "required": ["question", "answer"]
      }
    }
  ]

In [13]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = (
    prompt
    | model.bind(function_call={"name": "soccerfacts"}, functions= functions)
    | JsonOutputFunctionsParser()
)

In [14]:
chain.invoke(input={"soccer_player": "Mbappe"})

{'question': 'What is a curious fact about Kylian Mbappe?',
 'answer': 'Kylian Mbappe became the youngest French player to score in a World Cup at the age of 19 during the 2018 FIFA World Cup.'}

**Note:** OpenAI API has deprecated functions in favor of tools. See [here](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/openai_functions_agent/) for more info.

## Use of .bind() to attach OpenAI tools

**Note:** In the OpenAI Chat API, functions are now considered a legacy options that is deprecated in favor of tools. If you're creating agents using OpenAI LLM models, you should be using OpenAI Tools rather than OpenAI functions.

While you should generally use the .bind_tools() method for tool-calling models, you can also bind provider-specific args directly if you want lower level control:

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

In [ ]:
model = ChatOpenAI(model="gpt-3.5-turbo").bind(tools=tools)
model.invoke("What's the weather in SF, NYC and LA?")

## The assign() function allows adding keys to a chain
* Example: we will create a key name "operation_b" assigned to a custom function with a RunnableLambda.
* We will start with a very basic chain with just RunnablePassthrough:

In [20]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda

chain = RunnableParallel({"original_input": RunnablePassthrough()})

In [17]:
chain.invoke("whatever")

{'original_input': 'whatever'}

* As you can see, right now this chain is only assigning the user input to the "original_input" variable.
* Let's now add the new key "uppercase" with the assign function.
* In the new "uppercase" key, we will use a RunnableLambda with the custom function named `make_uppercase`

In [18]:
def make_uppercase(arg):
    return arg["original_input"].upper()

In [21]:
chain = RunnableParallel({"original_input": RunnablePassthrough()}).assign(uppercase=RunnableLambda(make_uppercase))

In [22]:
chain.invoke("whatever")

{'original_input': 'whatever', 'uppercase': 'WHATEVER'}

* As you can see, the output of the chain has now 2 keys: original_input and uppercase.
* In the uppercase key, we can see that the `make_uppercase` function has been applied to the user input.

## How to execute the code from Visual Studio Code
* In Visual Studio Code, see the file 006-builtin-functions-for-runnables.py
* In terminal, make sure you are in the directory of the file and run:
    * python 006-builtin-functions-for-runnables.py